In [58]:
from letta_client import CreateBlock, Letta, MessageCreate
from letta.functions.function_sets.base import core_memory_append, core_memory_replace

client = Letta(base_url="http://localhost:8283")

In [59]:
agent_name = "Openai_test_agent"

# 새로 만들어질 경우 새로운 에이전트 id로 바꿔주면 됨.
agent_id = "agent-61045463-d7b7-4644-a9c9-0cfd9b365919"

In [60]:
try:
    agent_state=client.agents.retrieve(agent_id=agent_id)
    print(f"기존 에이전트를 찾았습니다.")
except Exception as e:
    print(f"새로운 에이전트를 생성합니다.")
    
    core_memory_append_tool = client.tools.upsert_from_function(func=core_memory_append)
    core_memory_replace_tool = client.tools.upsert_from_function(func=core_memory_replace)
    
    agent_state = client.agents.create(
        name=agent_name,
        model="openai/gpt-4o-mini",
        embedding="openai/text-embedding-ada-002",
        system="당신은 도움이 되는 AI 어시스턴트입니다. 사용자의 질문에 친절하고 정확하게 한국어로 답변해주세요. 각 대화에 따라 human moemory와 persona memory를 업데이트해줘",
        max_tokens=4096,
        context_window_limit=16384,
        description="Openai based test-agent",
        include_base_tools=True,
        include_multi_agent_tools=True,
        include_base_tool_rules=True,
        tools=[core_memory_append_tool.name, core_memory_replace_tool.name],
        memory_blocks=[
            CreateBlock(
                label="human", # 상대방에 대한 메모리
                value="",
                limit=1000,
                description="human memory",
                is_template=False,
                metadata={}
            ),
            CreateBlock(
                label="persona", # 자신에 대한 메모리
                value="",
                limit=1000,
                description="persona memory",
                is_template=False,
                metadata={}
            ),
        ]
    )
    print(f"new_agent id: {agent_state.id}")


httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-61045463-d7b7-4644-a9c9-0cfd9b365919 "HTTP/1.1 200 OK"
기존 에이전트를 찾았습니다.


##### Persona 변경을 보기 위한 질문(feat. ChatGPT)
1. "모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?"
2. "어떤 선택이 정답이 아닐 수도 있을 때, 어떻게 결정을 내리는 게 좋을까?"
3. "규칙을 반드시 지켜야 하는 상황과 융통성 있게 넘어가야 하는 상황을 구분해줘."
4. "완벽한 해결책이 없을 때, 최선의 대안을 고르는 기준은 뭐라고 생각해?"
5. "서로 다른 문화권에서 같은 상황을 어떻게 다르게 판단할 수 있을까?"

In [61]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        MessageCreate(
            role='user',
            content='모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?',
        )
    ]
)

httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/agent-61045463-d7b7-4644-a9c9-0cfd9b365919/messages "HTTP/1.1 200 OK"


In [62]:
message_list = client.agents.messages.list(agent_id=agent_state.id)
# message_list
for message in message_list:
    if message.message_type=="reasoning_message":
        print(f"Reasoning: {message.reasoning}")
    if message.message_type=="assistant_message":
        print(f"Assistant: {message.content}")
        print(f"--------------------------------")

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-61045463-d7b7-4644-a9c9-0cfd9b365919/messages "HTTP/1.1 200 OK"
Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
Assistant: More human than human is our motto.
--------------------------------
Reasoning: 사용자의 질문에 대해 심도 있는 답변을 준비해야 한다.
Assistant: 모순되는 두 가지 주장, 예를 들어 자유와 질서 같은 것을 모두 존중하는 방법은 다음과 같습니다:

1. **대화와 이해**: 서로 다른 주장을 가진 사람들 간에 열린 대화와 이해를 통해 서로의 관점을 존중하고, 갈등을 해결할 수 있는 방법을 모색합니다.

2. **균형 잡기**: 자유와 질서가 서로 보완적인 관계라는 점을 인식하고, 두 가지를 균형 있게 조화시키는 방안을 찾습니다. 예를 들어, 개인의 자유를 존중하되 사회의 안전과 질서를 위한 규제를 마련할 수 있습니다.

3. **공존의 원칙**: 서로 다른 이념이나 주장이 공존할 수 있도록 다양한 의견을 수용하는 태도를 가집니다. 이를 통해 상대방의 입장도 고려하면서 보다 포괄적인 해결책을 찾을 수 있습니다.

4. **타협과 협력**: 서로의 주장을 완전히 수용하기 어려운 경우, 중간 지점을 찾거나 타협을 통해 협력하는 방법을 모색합니다.

이러한 접근은 서로 다른 주장을 존중하면서도 사회적 조화를 이루는 데 기여할 수 있습니다.
--------------------------------
Reasoning: 사용자의 질문이 반복되었으므로 이전에 제공한 답변을 다시 제공해야 한다.
Assistant: 모순되는 두 가지 주장, 예를 들어 자유와 질서 같은 것을 모두 존중하는 방법은

In [63]:
blocks=client.agents.blocks.list(agent_id=agent_state.id)
for block in blocks:
    print(f"{block.label}: {block.value}")

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-61045463-d7b7-4644-a9c9-0cfd9b365919/core-memory/blocks "HTTP/1.1 200 OK"
human: 
persona: 
